# Testing Non-Linear Products Valuation

This notebook walks through:

1. Loading historical index fixings  
2. Building a simple two-index yield curve  
3. Building **classic** SABR surfaces (no product split)  
4. Pricing IBOR caplets/floors and swaptions using classic Hagan SABR  
5. Building **top-down** SABR surfaces for SOFR (CAPLET vs SWAPTION)  
6. Pricing RFR caplets/swaptions using top-down SABR  


# 1. Imports & Setup

In [1]:
import pandas as pd
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.sabr import SabrModel
from fixedincomelib.valuation import IndexManager, ValuationEngineRegistry
from fixedincomelib.product import (
    ProductIborCapFloorlet, ProductOvernightCapFloorlet,
    ProductIborCapFloor,   ProductOvernightCapFloor,
    ProductIborSwaption,   ProductOvernightSwaption
)
from fixedincomelib.analytics import SABRCalculator
from fixedincomelib.data import DataCollection, Data1D, Data2D

print("Setup complete.")

Setup complete.


## 2) Load Index Fixings

On first call, `IndexManager.instance()` will read **`fixing/fixings.csv`** and populate its in-memory registry.

In [2]:
mgr = IndexManager.instance()
print("Loaded indices:", list(mgr._fixings.keys()))


Loaded indices: ['SOFR-1B', 'USD-LIBOR-3M', 'SONIA-1B']


## 3) Build Dummy Yield Curve

We use two indices (SOFR-1B and USD-LIBOR-BBA-1M) with piecewise-constant interpolation.


In [3]:
curve_data = [
    ["SOFR-1B",            "1M",  0.0020],
    ["SOFR-1B",            "3M",  0.0025],
    ["SOFR-1B",            "6M",  0.0030],
    ["USD-LIBOR-BBA-1M",   "1M",  0.0040],
    ["USD-LIBOR-BBA-1M",   "3M",  0.00425],
    ["USD-LIBOR-BBA-1M",   "6M",  0.00450],
]
df_curve = pd.DataFrame(curve_data, columns=["INDEX","AXIS1","VALUES"])

build_methods_curve = [
  {
    "TARGET":              "SOFR-1B",
    "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"
  },
  {
    "TARGET":              "USD-LIBOR-BBA-1M",
    "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"
  },
]


In [4]:
valueDate = "2025-06-26"

# start with an empty list
data_objs = []

# ZERO-RATE CURVES
for idx_name, sub in df_curve.groupby("INDEX"):
    d1 = Data1D.createDataObject(
        data_type       = "zero_rate",        # lowercase!
        data_convention = idx_name,
        df              = sub[["AXIS1","VALUES"]]
    )
    data_objs.append(d1)

dc = DataCollection(data_objs)

# 3) now build the curve from the DataCollection
yc = YieldCurve(valueDate, dc, build_methods_curve)
print("YieldCurve built. Components:", yc.components.keys())

ValueError: build_method must include a non-empty 'INSTRUMENTS' list.

## 4) Build Dummy SABR Model

We’ll calibrate a flat SABR surface for USD-LIBOR-BBA-1M so that our caplets can be priced.

In [ ]:
sabr_libor = pd.DataFrame([
    ["USD-LIBOR-BBA-1M", 0.25, 0.25, 0.015, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 0.25, 1.00, 0.017, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 0.25, 0.018, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 1.00, 0.020, 0.5, 0.2, -0.3],
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.25, 0.010, 0.5, 0.2, -0.1],
    ["SOFR-1B", 0.25, 1.00, 0.012, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 0.25, 0.013, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 1.00, 0.015, 0.5, 0.2, -0.1],
], columns=sabr_libor.columns)

sabr_data = pd.concat([sabr_libor, sabr_sofr], ignore_index=True)

In [ ]:
for idx_name, sub in sabr_data.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        d2 = Data2D.createDataObject(
            data_type       = param.lower(),   # e.g. "normalvol"
            data_convention = idx_name,
            df              = pivot
        )
        data_objs.append(d2)

# 6) Rebuild registry & classic‐SABR model
dc = DataCollection(data_objs)

In [ ]:
classic_build_methods = [
    {
      "TARGET":        idx,
      "VALUES":     param,
      "INTERPOLATION": "LINEAR",
      "SHIFT":         0.0,
      "VOL_DECAY_SPEED": 0.2
    }
    for idx in sabr_data["INDEX"].unique()
    for param in ["NORMALVOL","BETA","NU","RHO"]
]

classic_sabr = SabrModel.from_curve(
    valueDate             = valueDate,
    dataCollection        = dc,
    buildMethodCollection = classic_build_methods,
    ycModel               = yc
)
print("Classic SABR components:", classic_sabr.components.keys())


Classic SABR components: dict_keys(['USD-LIBOR-BBA-1M-NORMALVOL', 'USD-LIBOR-BBA-1M-BETA', 'USD-LIBOR-BBA-1M-NU', 'USD-LIBOR-BBA-1M-RHO', 'SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


## 5) Price with Classic SABR

In [ ]:
caplet_ibor = ProductIborCapFloorlet(
    startDate   ="2025-07-01",
    endDate     ="2025-10-01",
    index       ="USD-LIBOR-BBA-1M",
    optionType  ="CAP",
    strike      =0.02,
    notional    =1_000_000,
    longOrShort ="LONG"
)

ve1 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    caplet_ibor
)
ve1.calculateValue()
print("IBOR Caplet PV (classic):", ve1.value)

IBOR Caplet PV (classic): ['USD', np.float64(9.576384160647141e-07)]


In [ ]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1_000_000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve2 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve2.calculateValue()
print("IBOR Swaption PV (classic):", ve2.value)

IBOR Swaption PV (classic): ['USD', np.float64(21946381354.200516)]


In [ ]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1000000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve.calculateValue()
print("IBOR Swaption PV:", ve.value)

IBOR Swaption PV: ['USD', np.float64(21946381354.200516)]


## 6) Build Top-Down SABR Model for SOFR (CAPLET vs SWAPTION)


In [ ]:
sabr_sofr_cap = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.083333, 0.0090,    0.5,  0.20, -0.15],
    ["SOFR-1B", 0.25, 0.25,     0.0100,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.083333, 0.0110,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.25,     0.0120,    0.5,  0.20, -0.15],
], columns=sabr_sofr.columns)
sabr_sofr_cap["PRODUCT"] = "CAPLET"

sabr_sofr_sw = pd.DataFrame([
    ["SOFR-1B",0.25,1.0,0.012,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,2.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,3.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,4.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,1.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,2.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,3.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,4.0,0.016,0.5,0.2,-0.1]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr_sw["PRODUCT"] = "SWAPTION"

full_sabr_sofr = pd.concat([sabr_sofr_cap, sabr_sofr_sw], ignore_index=True)

data_objs = []

for idx_name, sub in df_curve.groupby("INDEX"):
    data_objs.append(
      Data1D.createDataObject("zero_rate", idx_name, sub[["AXIS1","VALUES"]])
    )

for idx_name, sub in full_sabr_sofr.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs.append(
          Data2D.createDataObject(param.lower(), idx_name, pivot)
        )

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc = DataCollection(data_objs)
td_sabr = SabrModel.from_data(
    valueDate             = valueDate,
    dataCollection        = dc,
    buildMethodCollection = topdown_build_methods,
    ycData                = df_curve,
    ycBuildMethods        = build_methods_curve
)

print("Top-down SABR components:", td_sabr.components.keys())

Top-down SABR components: dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


## 7) Price Top-Down RFR Products


In [ ]:
# 7a) Overnight caplet (top-down SABR)
caplet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve3 = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD":"top-down"},
    caplet_ois
)
ve3.calculateValue()
print("Overnight Caplet PV (top-down):", ve3.value)

Overnight Caplet PV (top-down): ['USD', np.float64(66.45271852624181)]


## 8) Valuation: Overnight Cap/Floor Wrapper


In [ ]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="FLOOR",
    strike       =0.016,
    notional     =1000000,
    longOrShort  ="SHORT"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD": "top-down"},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap/Floor PV:", ve.value)


Overnight Cap/Floor PV: ['USD', np.float64(-15124.237191922715)]


# 8) Price RFR caplets using bottom-up SABR

In [ ]:
corr_data = [
    [0.25, 0.25, 0.80],
    [0.25, 0.50, 0.78],
    [0.25, 1.00, 0.75],
    [1.00, 0.25, 0.78],
    [1.00, 0.50, 0.76],
    [1.00, 1.00, 0.74],
]
df_corr = pd.DataFrame(corr_data, columns=["EXPIRY","TENOR","CORR"])
df_corr["INDEX"] = "SOFR-1B"

In [ ]:
bu_objs = []
corr_pivot = (
    df_corr
    .pivot(index="EXPIRY", columns="TENOR", values="CORR")
    .sort_index(axis=0).sort_index(axis=1)
)
bu_objs.append(
    Data2D.createDataObject("corr", "SOFR-1B", corr_pivot)
)

In [ ]:
one_bd = 1.0/252

sabr_sofr_1bd = pd.DataFrame([
    ["SOFR-1B", 0.25,  one_bd,  0.0095,    0.50, 0.20, -0.15],
    ["SOFR-1B", 0.50,  one_bd,  0.0100,    0.50, 0.20, -0.15],
    ["SOFR-1B", 1.00,  one_bd,  0.0105,    0.50, 0.20, -0.15],
    ["SOFR-1B", 2.00,  one_bd,  0.0110,    0.50, 0.20, -0.15]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

for idx_name, sub in df_curve.groupby("INDEX"):
    bu_objs.append(
        Data1D.createDataObject("zero_rate", idx_name, sub[["AXIS1","VALUES"]])
    )

for idx_name, sub in sabr_sofr_1bd.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub
            .pivot(index="AXIS1", columns="AXIS2", values=param)
            .sort_index(axis=0).sort_index(axis=1)
        )
        bu_objs.append(
            Data2D.createDataObject(
                data_type       = param.lower(),
                data_convention = idx_name,
                df              = pivot
            )
        )

In [ ]:
bottom_up_build_methods_sofr = []
for param in ("NORMALVOL","BETA","NU","RHO"):
    bottom_up_build_methods_sofr.append({
      "TARGET":          "SOFR-1B",
      "VALUES":          param,
      "INTERPOLATION":   "LINEAR",
      "SHIFT":           0.0,
      "VOL_DECAY_SPEED": 0.2
    })

dc = DataCollection(bu_objs)
bottom_up_sofr_sabr = SabrModel.from_curve(
    valueDate,
    dc,
    bottom_up_build_methods_sofr,
    yc
)
print("Bottom-up 1BD‐tenor SABR components:", bottom_up_sofr_sabr.components.keys())


Bottom-up 1BD‐tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [ ]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)
ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)


Overnight Caplet PV (bottom-up): ['USD', np.float64(2.0804611554277008e-32)]


In [ ]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap PV (bottom-up):", ve.value)

Overnight Cap PV (bottom-up): ['USD', np.float64(2.635442270532238)]
